In [ ]:
import torch
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
path="../"
sys.path.append(path)
import Preprocessing as hsa_preprocessing
from sklearn.preprocessing import StandardScaler
import matplotlib.image as mpimg

from PIL import Image
from loguru import logger
import Model as hsa_model
import DataLoader as hsa_dataloader
import Viz as hsa_viz
from torch.utils.data import DataLoader
logger.add(sink=f"HSI_log.log",
            level="CRITICAL")

Read Images and plot shape of np.array associated with it. 

In [ ]:
image_path= "./night_sky2.jpeg"
img = Image.open(image_path)
array = np.array(img)
print(f"array shape: {array.shape}")

Plot images of the RGB spectra and original data as np before conversion to df. 

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10,10))
titles= ["Red Spectra", "Blue Spectra","Green Spectra", "Original Image"]
for i, ax in enumerate(axs.flat):
    try:
        sns.heatmap(array[:,:,i], square=True, xticklabels=False, yticklabels=False, cbar=False, ax=ax)
        ax.set_title(titles[i])
    except:
        ax.set_title(titles[i])
        image = mpimg.imread(image_path)
        ax.imshow(image)

Generate a list of data points from image and convert to df to begin vanilla preprocessing.

In [ ]:
data = None
columns = ["red", "green", "blue"]

for row in range(len(array)):
    try:
        data = np.append(data, array[row, : , :], axis=0)
    except:
        data = array[row, : , :]
 
scaler = StandardScaler()
scaler.fit(data)
preprocessed_df = pd.DataFrame(scaler.transform(data), columns=columns)

Instantiate the model and dataloader with preprocessed data as a np.array()

In [ ]:
anomaly_std_toll= 1
penalty_ratio=.3
cutoff_dist=2
model = hsa_model.HSI_model(
            penalty_ratio=penalty_ratio,
            cutoff_dist= cutoff_dist,
            converge_toll= 1e-5,
            anomaly_std_toll= anomaly_std_toll,
            affinity_matrix_iterations= 20,
            lr= 2.7,
            logger=logger,
            multifilter_flag=0,
        )
dataset = hsa_dataloader.HSI_dataset(preprocessed_np=preprocessed_df.to_numpy(), logger=logger)
dataloader = DataLoader(dataset, batch_size=10)


In [ ]:
base_directory = "Images/"
log_directory = f"{base_directory}/logs/"
results_directory = f"{base_directory}/results/"
model.set_directories(log_directory, results_directory)

logger.info("Starting to run through the dataloader on initial pass.")
num_samples = 1000
unique_id_str = "test"
iterations = 10000

for i, data in enumerate(dataloader):  # setting up gpus
    model.set_trial(i * num_samples, num_samples, unique_id_str)
    # Model set up and weight generation
    model.readData(
        data_multifilter_df=data.squeeze(0)
    ).vertWeights_distances().affinityGen().graphEvo()
    # Training steps
    model.torch_optimize_POF(iterations=iterations)
    # Prediction step
    model.model_Predictions(preprocessed_df)
    # Store anomalous predictions throughout all batches for use in multi filter
    try:
        total_anomaly_index = np.append(total_anomaly_index, model.x_label)
    except: 
        total_anomaly_index = model.x_label
        print(f"only on 0 :{i}")
    if i > 1000:
        print(i)
        break
        

In [ ]:
mix_index, mix_data, anomaly_index = model.global_collect_multifilter_df(
            preprocessed_df.to_numpy(),
            total_anomaly_index[: len(preprocessed_df.to_numpy())].astype(int),
            mf_num_samples=9 * len(total_anomaly_index),
        )
anomaly_pred_freq_df = pd.DataFrame()
anomaly_pred_freq_df["User DF Index"] = anomaly_index
anomaly_pred_freq_df.set_index("User DF Index")
anomaly_pred_freq_df["Anom Pred Count"] = np.zeros(len(anomaly_index))

In [ ]:
model.uni_shuffle_multifilter_df(
            mix_index.astype(int), mix_data.astype(int), anomaly_index.astype(int)
        )
mf_data = model.all_data
logger.debug(
    "Anomalous data has been colleted into first multifilter dataset."
)

In [ ]:
print(f"Original data input- preprocessed_df: {preprocessed_df.shape} \nFirst rank anomlus predictions- anomaly_pred_freq_df:{anomaly_pred_freq_df.shape} \nMultifilter data shape- mf_data: {mf_data.shape}")

In [ ]:
multi_filters = 5
user_location = model.all_index_user

for i in range(multi_filters):
    batch_dataset = hsa_dataloader.HSI_dataset(mf_data, logger)
    batch_loader = DataLoader(
        batch_dataset, batch_size=num_samples, 
    )
    j = 0
    for data in batch_loader:
        print(data.shape)
        # # Set up multi filter model
        MF_model = hsa_model.HSI_model(
            penalty_ratio=penalty_ratio,
            cutoff_dist= cutoff_dist,
            converge_toll= 1e-5,
            anomaly_std_toll= anomaly_std_toll,
            affinity_matrix_iterations= 20,
            lr= 2.7,
            logger=logger,
            multifilter_flag=1,
        )
        MF_model.set_directories(
            log_directory, results_directory
        )
        MF_model.set_trial(j * len(data), len(data), unique_id_str)
        MF_model.readData(
            data_multifilter_df=data.squeeze(0)
        ).vertWeights_distances().affinityGen().graphEvo()

        # # Train MF_MODEL
        MF_model.torch_optimize_POF(iterations=iterations)
        MF_model.model_Predictions(
            preprocessed_df, multifilter_flag=1, user_location=user_location
        )
        j += 1
        if j > 3:
            break
        anomaly_pred_freq_df.loc[
            anomaly_pred_freq_df["User DF Index"].isin(MF_model.x_label),
            "Anom Pred Count",
        ] += 1

        logger.trace(
            f"Multifilter {i} of {multi_filters} multifilters is complete."
        )
        # # Global multifilter
        mix_index, mix_data, anomaly_index = (
            model.global_collect_multifilter_df(
                preprocessed_df.to_numpy(),
                total_anomaly_index[: len(preprocessed_df.to_numpy())].astype(int),
                mf_num_samples=9 * len(total_anomaly_index),
            )
        )
        MF_model.uni_shuffle_multifilter_df(mix_index, mix_data, anomaly_index)
        mf_data = MF_model.all_data
        user_location = MF_model.all_index_user

In [ ]:
viz = hsa_viz.HSI_viz(
    MF_model.m,
    MF_model.preprocessed_np,
    num_samples,
    0,
    0,
    1,
    0,
    "plot_directory",
    "unique_id_str",
    logger,
)
viz.heatmap_bin_predictions_vert(
    MF_model.bin_score,
    MF_model.x_ticks,
    MF_model.x_label,
)


In [ ]:
preprocessed_df["Anom Pred Count"] = np.zeros(len(preprocessed_df))
for i in anomaly_pred_freq_df[anomaly_pred_freq_df["Anom Pred Count"]>0].index:
    preprocessed_df.loc[anomaly_pred_freq_df.loc[i], "Anom Pred Count"] = anomaly_pred_freq_df.loc[i,"Anom Pred Count"]  

In [ ]:
image_width = 1280
anomaly_score_np = []
for i in range(int(len(preprocessed_df)/image_width)):
    anomaly_score_np.append(preprocessed_df["Anom Pred Count"][i*image_width:(1+i)*image_width])

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10,10))
titles= ["Red Spectra", "Blue Spectra","Green Spectra", "Anomaly Scores"]
for i, ax in enumerate(axs.flat):
    try:
        sns.heatmap(array[:,:,i], square=True, xticklabels=False, yticklabels=False, cbar=False, ax=ax)
        ax.set_title(titles[i])
    except:
        ax.set_title(titles[i])
        sns.heatmap(anomaly_score_np, square=True, xticklabels=False, yticklabels=False, cbar=False, ax=ax)
        

In [ ]:
plt.spy(anomaly_score_np)